databricks notebook7: create Trip fact table

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
trip_df = spark.read.format('delta').load('abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/trip_transactions') \
    .withColumn('StartDate',to_date('trip_start_timestamp')) \
    .withColumn('EndDate', to_date('trip_end_timestamp'))

In [ ]:
rating_df = spark.read.format('delta').load('abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/trip_transactions')

In [ ]:
# register them to temporary views
trip_df.createOrReplaceTempView('trip_view')
rating_df.createOrReplaceTempView('rating_view')
spark.read.format("delta").load("abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_customer").createOrReplaceTempView("dim_customer")
spark.read.format("delta").load("abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_driver").createOrReplaceTempView("dim_driver")
spark.read.format("delta").load("abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_location").createOrReplaceTempView("dim_location")
spark.read.format("delta").load("abfss://trip-data@<your-storage-account>.dfs.core.windows.net/silver/dim_date").createOrReplaceTempView("dim_date")

In [ ]:
%sql
select * from trip_view limit 10

In [ ]:
# build fact table
%sql

create or replace table fact_trip as select 
    t.trip_id,
    c.KeycustomerID,
    d.KeydriverID,
    p.KeylocationID as PickUpLocationID,
    q.KeylocationID as DropOffLocationID,
    sd.KeydateID as StartDateID,
    ed.KeydateID as EndDateID,
    t.total_distance,
    t.total_fare,
    t.delay_start_time_mins as delay_mins,
    r.customer_ratings,
    r.driver_ratings
from trip_view t 
left join dim_customer c on t.customerID = c.customerID
left join dim_driver d on t.driverID = d.driverID
left join dim_location p on t.source_location_addtress1 = p.addtress
                        and t.source_city = p.city 
                        and t.source_country = p.country 
                        and p.location_type = 'pickup'
left join dim_location q on t.destination_location_addtress1 = q.addtress 
                        and t.destination_city = q.city 
                        and t.destination_country = q.country 
                        and q.location_type = 'dropoff'
left join dim_date sd on t.StartDate = sd.Date 
                    and t.EndDate = sd.Date 
left join rating_view r on t.trip_id = r.trip_id 
                        and t.customer_id = r.customer_id
                        and t.driver_id = r.driver_id 